In [1]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_10_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-17 22:16:05.364248


In [2]:
%matplotlib qt

# Posner A

In [3]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_10_02/m_10_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 14', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_10_02/10_2a_posner_task_2023-06-01_15h04.36.005/10_2a_posner_task_2023-06-01_15h04.36.005.csv
Found '2b' in file: m_10_02/10_2b_posner_task_2023-06-01_15h55.31.572/10_2b_posner_task_2023-06-01_15h55.31.572.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 14', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-06-01 15:06:50.392875+00:00 - 2023-06-01 15:19:36.072875+00:00
  [0] Keeping  (2023-06-01 15:06:50.392875+00:00 - 2023-06-01 15:06:50.392875+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-01 15:07:43.212875+00:00 - 2023-06-01 15:07:43.212875+00:00 -> 52.82 - 52

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


In [4]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
413 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 413 events and 701 original time points ...
4 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [5]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Fitting ICA took 612.6s.


Method,fastica
Fit,1000 iterations on raw data (667993 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [6]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=667993
    Range : 81094 ... 749086 =     81.094 ...   749.086 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
333 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
333 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
333 matching events found
No baseline correction applied
0 projection items activated


In [7]:
print(ica.exclude)

[43, 41, 49, 24, 22, 20, 26, 29, 28, 30, 38, 35, 36, 0, 7, 16, 15, 18, 19]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 19 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_10_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_10_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_10_02/m_10_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_10_02
├── m_10_02_pos2a.eeg
├── m_10_02_pos2a.vhdr
└── m_10_02_pos2a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_10_02/m_10_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_10_02/10_2a_posner_task_2023-06-01_15h04.36.005/10_2a_posner_task_2023-06-01_15h04.36.005.csv
Found '2b' in file: m_10_02/10_2b_posner_task_2023-06-01_15h55.31.572/10_2b_posner_task_2023-06-01_15h55.31.572.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-06-01 15:58:03.274160+00:00 - 2023-06-01 16:10:22.634160+00:00
  [0] Keeping  (2023-06-01 15:58:03.274160+00:00 - 2023-06-01 15:58:03.274160+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-06-01 15:58:19.834160+00:00 - 2023-06-01 15:58:19.834160+00:00 -> 16.56 - 16

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [388] Keeping  (2023-06-01 16:09:23.892160+00:00 - 2023-06-01 16:09:23.892160+00:00 -> 680.618 - 680.618)
  [389] Keeping  (2023-06-01 16:09:27.272160+00:00 - 2023-06-01 16:09:27.272160+00:00 -> 683.998 - 683.998)
  [390] Keeping  (2023-06-01 16:09:27.372160+00:00 - 2023-06-01 16:09:27.372160+00:00 -> 684.098 - 684.098)
  [391] Keeping  (2023-06-01 16:09:28.902160+00:00 - 2023-06-01 16:09:28.902160+00:00 -> 685.628 - 685.628)
  [392] Keeping  (2023-06-01 16:09:30.402160+00:00 - 2023-06-01 16:09:30.402160+00:00 -> 687.128 - 687.128)
  [393] Keeping  (2023-06-01 16:09:32.982160+00:00 - 2023-06-01 16:09:32.982160+00:00 -> 689.708 - 689.708)
  [394] Keeping  (2023-06-01 16:09:33.082160+00:00 - 2023-06-01 16:09:33.082160+00:00 -> 689.808 - 689.808)
  [395] Keeping  (2023-06-01 16:09:35.412160+00:00 - 2023-06-01 16:09:35.412160+00:00 -> 692.138 - 692.138)
  [396] Keeping  (2023-06-01 16:09:36.912160+00:00 - 2023-06-01 16:09:36.912160+00:00 -> 693.638 - 693.638)
  [397] Keeping  (2023-06-01

In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
414 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 414 events and 701 original time points ...
5 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-06-01 15:58:55.658160+00:00 - 2023-06-01 16:10:04.558160+00:00
  [0] Keeping  (2023-06-01 15:58:55.658160+00:00 - 2023-06-01 15:58:55.658160+00:00 -> 52.384 - 52.384)
  [1] Keeping  (2023-06-01 15:58:57.158160+00:00 - 2023-06-01 15:58:57.158160+00:00 -> 53.884 - 53.884)
  [2] Keeping  (2023-06-01 15:58:59.688160+00:00 - 2023-06-01 15:58:59.688160+00:00 -> 56.414 - 56.414)
  [3] Keeping  (2023-06-01 15:58:59.788160+00:00 - 2023-06-01 15:58:59.788160+00:00 -> 56.514 - 56.514)
  [4] Keeping  (2023-06-01 15:59:02.158160+00:00 - 2023-06-01 15:59:02.158160+00:00 -> 58.884 - 58.884)
  [5] Keeping  (2023-06-01 15:59:03.658160+00:00 - 2023-06-01 15:59:03.658160+00:00 -> 60.384 - 60.384)
  [6] Keeping  (2023-06-01 15:59:07.148160+00:00 - 2023-06-01 15:59:07.148160+00:00 -> 63.874 - 63.874)
  [7] Keeping  (2023-06-01 15:59:07.248160+00:00 - 2023-06-01 15:59

Method,fastica
Fit,67 iterations on raw data (668900 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [15]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=668900
    Range : 52384 ... 721283 =     52.384 ...   721.283 secs
Ready.
Using pyopengl with version 3.1.6


Traceback (most recent call last):
  File "/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py", line 4402, in keyPressEvent
    slot()
  File "/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py", line 4244, in _toggle_epoch_histogram
    fig = self._create_epoch_histogram()
  File "/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/mne/viz/_figure.py", line 550, in _create_epoch_histogram
    ptp = np.ptp(epochs.get_data(), axis=2)
  File "<__array_function__ internals>", line 200, in ptp
  File "/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 2695, in ptp
    return _methods._ptp(a, axis=axis, out=out, **kwargs)
  File "/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/numpy/core/_methods.py", line 283, in _ptp
    umr_maximum(a, axis, None, out, keepdims),
numpy.AxisError: axis 2 is out of bounds for array of dimensi

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
334 matching events found
No baseline correction applied
0 projection items ac

In [16]:
ica_b.exclude

[42, 43, 58, 53, 23, 24, 25, 27, 35, 36, 0, 2, 7, 13, 14, 15]

In [17]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 16 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [18]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_10_02/m_10_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [19]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_10_02
├── m_10_02_pos2a.eeg
├── m_10_02_pos2a.vhdr
├── m_10_02_pos2a.vmrk
├── m_10_02_pos2b.eeg
├── m_10_02_pos2b.vhdr
└── m_10_02_pos2b.vmrk

1 directory, 6 files


# ...

In [20]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b